<a href="https://colab.research.google.com/github/couragedike1/-Recommendation-Systems/blob/main/Renforcement_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Install required libraries in Google Colab
!pip install gym numpy
!apt-get install -y xvfb python-opengl ffmpeg

import gym
import numpy as np
import time


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-opengl


In [2]:
# Set up the CartPole environment
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]  # 4 for CartPole (position, velocity, angle, angular velocity)
action_size = env.action_space.n  # 2 actions (left, right)

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# Discretize the continuous state space (simple bucketing)
def discretize_state(state, bins=(10, 10, 10, 10)):
    bounds = [(-2.4, 2.4), (-3.0, 3.0), (-0.5, 0.5), (-2.0, 2.0)]  # Approximate bounds for CartPole states
    discretized = []
    for i in range(len(state)):
        scaling = (state[i] - bounds[i][0]) / (bounds[i][1] - bounds[i][0])
        new_state = int(np.clip(np.round(scaling * (bins[i] - 1)), 0, bins[i] - 1))
        discretized.append(new_state)
    return tuple(discretized)

In [4]:
# Initialize Q-table
q_table = np.zeros((10, 10, 10, 10, action_size))  # 10 bins per state dimension, 2 actions

In [5]:
# Hyperparameters
learning_rate = 0.1
discount_factor = 0.99
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995
min_epsilon = 0.01
episodes = 1000
max_steps = 200

In [7]:
# Training loop
for episode in range(episodes):
    state = env.reset()
    state = discretize_state(state)
    total_reward = 0

    for step in range(max_steps):
        # Choose action (epsilon-greedy)
        if np.random.rand() < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(q_table[state])  # Exploit

        # Take action
        next_state, reward, done, _ = env.step(action)
        next_state = discretize_state(next_state)
        total_reward += reward

        # Update Q-table
        best_next_action = np.argmax(q_table[next_state])
        q_table[state][action] += learning_rate * (
            reward + discount_factor * q_table[next_state][best_next_action] - q_table[state][action]
        )

        state = next_state

        if done:
            break

    # Decay epsilon
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # Print progress every 100 episodes
    if (episode + 1) % 100 == 0:
      print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {epsilon:.3f}")

Episode 100/1000, Total Reward: 24.0, Epsilon: 0.606
Episode 200/1000, Total Reward: 13.0, Epsilon: 0.367
Episode 300/1000, Total Reward: 20.0, Epsilon: 0.222
Episode 400/1000, Total Reward: 30.0, Epsilon: 0.135
Episode 500/1000, Total Reward: 28.0, Epsilon: 0.082
Episode 600/1000, Total Reward: 66.0, Epsilon: 0.049
Episode 700/1000, Total Reward: 111.0, Epsilon: 0.030
Episode 800/1000, Total Reward: 74.0, Epsilon: 0.018
Episode 900/1000, Total Reward: 47.0, Epsilon: 0.011
Episode 1000/1000, Total Reward: 23.0, Epsilon: 0.010


In [10]:
# Test the trained agent
env = gym.make('CartPole-v1', render_mode='human')  # Render in Colab (might need additional setup)
state = env.reset()
state = discretize_state(state)
done = False
total_reward = 0

print("Testing trained agent...")


Testing trained agent...


/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [13]:
for step in range(max_steps):
    env.render()  # Display the environment
    action = np.argmax(q_table[state])
    next_state, reward, done, _ = env.step(action)
    next_state = discretize_state(next_state)
    total_reward += reward
    state = next_state
    time.sleep(0.05)  # Slow down for visibility
    if done:
        break

print(f"Test Episode Reward: {total_reward}")

Test Episode Reward: 30.0
